# Apprentissage automatique

## TP N°3 - Apprentissage automatique
### Le Perceptron multicouche

### 1 - Chargement des données de MNIST

In [3]:
# Insertion des bibliothèques
import mnist
import numpy as np

In [4]:
print("La version de numpy est :", np.__version__)

La version de numpy est : 1.26.4


In [5]:
# Data loading

mnistData = mnist.MNIST("Data\\MNIST") # Path of the four ubyte files.
trainData, trainLabels = mnistData.load_training()
testData, testLabels = mnistData.load_testing()
trainData = np.array(trainData)
trainLabels = np.array(trainLabels).reshape(len(trainLabels), 1)
testData = np.array(testData)
testLabels = np.array(testLabels).reshape(1, len(testLabels))

In [6]:
# Visualisation du contenu d’une image 

print(mnist.MNIST().display(trainData[7])) # I is the index of the image et peut être n'importe quel chifre


............................
............................
............................
............................
............................
..............@@@@@@........
...........@@@@@@@@@@.......
..........@@@@@@@@@@@@......
..........@@@.....@@@@......
..................@@@.......
..................@@@.......
..................@@@.......
...............@@@@@@.......
..............@@@@@.........
.........@@@@@@@@@..........
.........@@@@@@@@@@.........
.................@@.........
.................@@.........
.................@@.........
................@@@.........
.......@.......@@@@.........
.....@@@.....@@@@...........
.....@@@@@@@@@@@............
.......@@@@@@@..............
........@@..................
............................
............................
............................


Le résultat obtenu n'est pas fiable/correcte. Une modification est nécessaire.

### 2 Normalisation des données

La méthode d’optimisation liée au Perceptron multicouche (c.-à-d. la descente de gradient),
est optimale lorsque les données sont dans un interval particulier Xn ∈ [0, 1]. Ainsi, normalisez
les données des bases WDBC et MNIST par leur maximum.

In [7]:
# Data loading

mnistData = mnist.MNIST("Data\\MNIST") # Path of the four ubyte files.
trainData, trainLabels = mnistData.load_training()
testData, testLabels = mnistData.load_testing()
trainData = np.array(trainData)/255 # afin de diviser par des floatant toujours entre 0 et 1
trainLabels = np.array(trainLabels).reshape(len(trainLabels), 1)
testData = np.array(testData)/255
testLabels = np.array(testLabels).reshape(1, len(testLabels))

In [8]:
trainLabelsMatrix = np.zeros((trainLabels.size, 10))
for n in range(trainLabels.size):
    trainLabelsMatrix[n, trainLabels[n]] = 1
    
# return trainData, trainLabelsMatrix, testData, testLabels

In [9]:
# Visualisation du contenu d’une image 

print(mnist.MNIST().display(trainData[4])) # I is the index of the image et peut être n'importe quel chifre


............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................


In [10]:
def readMnist(mnistPath) : 
    mnistData = mnist.MNIST(mnistPath) # Path of the four ubyte files.
    trainData, trainLabels = mnistData.load_training()
    testData, testLabels = mnistData.load_testing()
    trainData = np.array(trainData)/255 # afin de diviser par des floatant toujours entre 0 et 1
    trainLabels = np.array(trainLabels).reshape(len(trainLabels), 1)
    testData = np.array(testData)/255
    testLabels = np.array(testLabels).reshape(1, len(testLabels))
    trainLabelsMatrix = np.zeros((trainLabels.size, 10))
    for n in range(trainLabels.size):
        trainLabelsMatrix[n, trainLabels[n]] = 1
        
    return trainData, trainLabelsMatrix, testData, testLabels

trainData, trainLabels, testData, testLabels = readMnist("Data\\MNIST")
print(trainData[12, :])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

### 3 Fonction d’activation

Afin que le réseau de neurones apprenne, il est nécessaire de définir une fonction d’activation.
Dans notre cas, nous utiliserons la fonction sigmoïde, voir équation 1 ainsi que sa dérivée pour
l’étape de back-propagation, voir équation 2. Où derivate vaut False si on souhaite calculer f(x) et True si on souhaite calculer f′(x).

In [11]:
def sigmoid(x, derivate=False):
    fx = 1/(1+ np.exp(-x))
    if derivate:
        return fx*(1 - fx)
    return fx

### 4 Apprentissage du réseau de neurones

Dans cette partie, on souhaite définir le réseau de neurones et l’entraîner sur les données d’entraînement. Pour un meilleur apprentissage, nous utiliserons la technique du batch-processing.

Notre réseau de neurones sera défini comme suit :

• Nombre de neurones d’entrées : 3 pour WDBC - 28 ∗ 28 = 784 pour MNIST.

• Nombre de couches cachées : 1.

• Nombre de neurones dans le niveau caché : 20 pour WDBC - 30 pour MNIST.

• Nombre de neurones de sorties : 1 pour WDBC - 10 pour MNIST.


Afin de créer ce réseau, définissez la fonction suivante :

In [12]:
def trainNeuralNetwork(input, output, hiddenLayerSize, batchSize, learningRate,numberOfIterations):
    """ Train neural network"""
    inputSize = input.shape[1]
    outputSize = output.shape[1]
    errors = np.zeros(numberOfIterations)
    
    # Initialisation des poids
    weigths0 = np.random.rand(inputSize, hiddenLayerSize)/batchSize
    weigths1 = np.random.rand(hiddenLayerSize, outputSize)/batchSize
    
    # Algorithme
    for i in range(numberOfIterations):
        # Calcul de la sortie de chaque couche 
        # Y l = f(qN i=1 WilXil)
        layer0 = input
        layer1 = sigmoid(layer0.dot(weigths0))
        layer2 = sigmoid(layer1.dot(weigths1))
        
        
        # Calcul de l’erreur d’apprentissage El
        layer2Error = output - layer2
        errors[i] = np.mean(np.sum(abs(layer2Error),1))
        
        # Rétropropagation de l’erreur par descente de gradient
        layer2Delta = layer2Error*sigmoid(layer1.dot(weigths1),True)
        layer1Error = layer2Delta.dot(weigths1.transpose())
        layer1Delta = layer1Error*sigmoid(layer0.dot(weigths0),True)
        
        # Mise à jour des poids Wli
        weigths0 += learningRate*layer0.transpose().dot(layer1Delta)
        weigths1 += learningRate*layer1.transpose().dot(layer2Delta)
    
    return weigths0,weigths1,errors

In [15]:
trainData, trainLabels, testData, testLabels = readMnist("Data\\MNIST")
trainNeuralNetwork(trainData, trainLabels, 30 ,50 , 10,5)

C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_11816\1238430488.py:2: RuntimeWarning: overflow encountered in exp
  fx = 1/(1+ np.exp(-x))


(array([[0.00663217, 0.0130846 , 0.00871711, ..., 0.0117892 , 0.00577801,
         0.00245706],
        [0.00151639, 0.01087824, 0.01748484, ..., 0.00269502, 0.0036236 ,
         0.00932723],
        [0.00088593, 0.01404495, 0.01458021, ..., 0.01293558, 0.00181888,
         0.00230094],
        ...,
        [0.01643923, 0.01331976, 0.01357155, ..., 0.00827732, 0.01626867,
         0.00290808],
        [0.00732486, 0.00935086, 0.01122024, ..., 0.00045211, 0.01729031,
         0.01611908],
        [0.01755922, 0.00569855, 0.00072022, ..., 0.00354022, 0.01819503,
         0.01903159]]),
 array([[-49011.38676352, -49412.49144745, -48217.51956861,
         -48752.63287559, -50233.16012494, -49341.46809022,
         -49823.02691954, -48382.57860896, -49112.09601438,
         -49884.73989987],
        [-49025.17440268, -49609.85856465, -48361.73236918,
         -48907.25032662, -50382.11764237, -49435.58983981,
         -49920.6839502 , -48488.60991787, -49257.44573935,
         -50046.806075

In [17]:
def trainNeuralNetwork(input, output, hiddenLayerSize, batchSize, learningRate,numberOfIterations):
    """ Train neural network"""
    inputSize = input.shape[1]
    outputSize = output.shape[1]
    errors = np.zeros(numberOfIterations)
    
    # Initialisation des poids
    weigths0 = np.random.rand(inputSize, hiddenLayerSize)/batchSize
    weigths1 = np.random.rand(hiddenLayerSize, outputSize)/batchSize
    
    # Algorithme
    for i in range(numberOfIterations):
        n = np.random.randint(0,input.shape[0] - batchSize +1)
        # Calcul de la sortie de chaque couche 
        # Y l = f(qN i=1 WilXil)
        layer0 = input[n:n + batchSize, :]
        layer1 = sigmoid(layer0.dot(weigths0))
        layer2 = sigmoid(layer1.dot(weigths1))
        
        
        # Calcul de l’erreur d’apprentissage El
        layer2Error = output[n:n + batchSize, :] - layer2
        errors[i] = np.mean(np.sum(abs(layer2Error),1))
        
        # Rétropropagation de l’erreur par descente de gradient
        layer2Delta = layer2Error*sigmoid(layer1.dot(weigths1),True)
        layer1Error = layer2Delta.dot(weigths1.transpose())
        layer1Delta = layer1Error*sigmoid(layer0.dot(weigths0),True)
        
        # Mise à jour des poids Wli
        weigths0 += learningRate*layer0.transpose().dot(layer1Delta)
        weigths1 += learningRate*layer1.transpose().dot(layer2Delta)
    
    return weigths0,weigths1,errors

In [18]:
trainData, trainLabels, testData, testLabels = readMnist("Data\\MNIST")
trainNeuralNetwork(trainData, trainLabels, 30 ,50 , 10,5)

(array([[0.00338406, 0.01745264, 0.01210508, ..., 0.01244565, 0.01839823,
         0.00518219],
        [0.00891415, 0.0038443 , 0.00054418, ..., 0.013169  , 0.00866591,
         0.01283   ],
        [0.01266213, 0.01366004, 0.00105164, ..., 0.01135148, 0.00211904,
         0.01083788],
        ...,
        [0.01682238, 0.00129871, 0.0009608 , ..., 0.012013  , 0.01582489,
         0.01713862],
        [0.00845709, 0.0118861 , 0.01454131, ..., 0.01844406, 0.01962531,
         0.0093746 ],
        [0.01481772, 0.00588983, 0.00887673, ..., 0.01470143, 0.00804727,
         0.00545715]]),
 array([[-41.53497115, -33.44446543, -47.5712246 , -36.34823277,
         -40.78514382, -44.55927826, -34.04218305, -38.3930395 ,
         -41.60184188, -45.58233091],
        [-41.35715688, -33.39467011, -47.37929442, -36.10426717,
         -40.73259579, -44.40138094, -34.05856795, -38.09873018,
         -41.38432844, -45.34710703],
        [-41.43761364, -33.84389031, -47.6726482 , -36.53504699,
        